## import包

In [1]:
import sys
sys.path.append('../AutoX')

In [2]:
import datetime
import pandas as pd
import os
from autox.autox_recommend.recall_and_rank.feature_engineer import feature_engineer
from autox.autox_recommend.recall_and_rank.ranker import ranker, ranker_test, inference
from autox.autox_recommend.recall_and_rank.recalls import binary_recall
from autox.autox_recommend.recall_and_rank.recalls import history_recall
from autox.autox_recommend.recall_and_rank.recalls import itemcf_recall
from autox.autox_recommend.recall_and_rank.recalls import popular_recall
from autox.autox_recommend.recall_and_rank.recalls import w2v_concent_recall
from autox.autox_recommend.metrics import mapk

/home/caihengxing/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/caihengxing/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not mo

In [3]:
# from autox import AutoXRecommend

In [4]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## 读取数据

In [5]:
path = '~/AutoX/autox/autox_recommend/datasets/MovieLens_AutoX/'

In [6]:
inter_df = pd.read_csv(path + 'inter_df.csv')
item_df = pd.read_csv(path + 'item_df.csv')
test = pd.read_csv(path + 'test.csv')

## 配置参数

In [7]:
uid = 'userId'
iid = 'movieId'
time_col = 'time'
recall_num = 100

## 准备测试集结果

In [8]:
inter_df[time_col].min(), inter_df[time_col].max()

('1995-01-09 11:46:49', '2019-11-14 23:20:55')

In [9]:
test[time_col].min(), test[time_col].max()

('2019-11-15 00:08:42', '2019-11-21 09:06:53')

In [10]:
assert(test[time_col].min() > inter_df[time_col].max())

In [11]:
positive_items_test = test.groupby([uid])[iid].apply(list)
test_users = positive_items_test.keys()
test_items = []

for i, user in tqdm(enumerate(test_users)):
    test_items.append(positive_items_test[user])
    
print("Total users in testidation:", len(test_users))

283it [00:00, 175071.98it/s]

Total users in testidation: 283


## 时间列转化

In [12]:
inter_df[time_col] = pd.to_datetime(inter_df[time_col])

In [13]:
inter_df.head()

,userId,movieId,time
0,1,296,2006-05-17 15:34:04
1,1,307,2006-05-17 12:27:08
2,1,665,2006-05-17 15:13:40
3,1,1237,2006-05-17 12:27:19
4,1,2632,2006-05-17 15:04:08


## 执行AutoX

In [14]:
class AutoXRecommend():
    def __init__(self):
        pass

    def fit(self, inter_df, user_df, item_df,
                  uid, iid, time_col,
                  recall_num,
                  time_decay=0.8,
                  debug=False, debug_save_path=None):

        self.inter_df = inter_df
        self.user_df = user_df
        self.item_df = item_df

        self.uid = uid
        self.iid = iid
        self.time_col = time_col
        self.recall_num = recall_num
        self.time_decay = time_decay
        self.debug = debug

        if debug:
            assert debug_save_path is not None
            path_output = debug_save_path
            self.path_output = path_output
            os.makedirs(path_output, exist_ok=True)

        temp_date = datetime.datetime.strptime(str(inter_df[time_col].max()), '%Y-%m-%d %H:%M:%S') + \
                    datetime.timedelta(days=1)
        valid_date = str(datetime.datetime(temp_date.year, temp_date.month, temp_date.day))
        self.valid_date = valid_date

        train_date = datetime.datetime.strptime(valid_date, '%Y-%m-%d %H:%M:%S') - datetime.timedelta(days=7)
        train_date = str(train_date)


        print('\npopular_recall')
        print('train')
        popular_recall_train = popular_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('valid')
        popular_recall_valid = popular_recall(None, inter_df, date=valid_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')

        print('\nhistory_recall')
        print('train')
        history_recall_train = history_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('valid')
        history_recall_valid = history_recall(None, inter_df, date=valid_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')

        print('\nitemcf_recall')
        print('train')
        if os.path.exists(f'{path_output}/itemcf_recall_train.hdf'):
            itemcf_recall_train = pd.read_hdf(f'{path_output}/itemcf_recall_train.hdf')
        else:
            itemcf_recall_train = itemcf_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train',
                                                topk=1000, use_iif=False, sim_last_days=14,
                                                time_decay=time_decay)
        print('valid')
        if os.path.exists(f'{path_output}/itemcf_recall_valid.hdf'):
            itemcf_recall_valid = pd.read_hdf(f'{path_output}/itemcf_recall_valid.hdf')
        else:
            itemcf_recall_valid = itemcf_recall(None, inter_df, date=valid_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train',
                                                topk=1000, use_iif=False, sim_last_days=14,
                                                time_decay=time_decay)
        if debug:
            itemcf_recall_train.to_hdf(f'{path_output}/itemcf_recall_train.hdf', 'w', complib='blosc', complevel=5)
            itemcf_recall_valid.to_hdf(f'{path_output}/itemcf_recall_valid.hdf', 'w', complib='blosc', complevel=5)

        print('\nbinary_recall')
        print('train')
        if os.path.exists(f'{path_output}/binary_recall_train.hdf'):
            binary_recall_train = pd.read_hdf(f'{path_output}/binary_recall_train.hdf')
        else:
            binary_recall_train = binary_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train', topk=1000)

        print('valid')
        if os.path.exists(f'{path_output}/binary_recall_valid.hdf'):
            binary_recall_valid = pd.read_hdf(f'{path_output}/binary_recall_valid.hdf')
        else:
            binary_recall_valid = binary_recall(None, inter_df, date=valid_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train', topk=1000)
        if debug:
            binary_recall_train.to_hdf(f'{path_output}/binary_recall_train.hdf', 'w', complib='blosc', complevel=5)
            binary_recall_valid.to_hdf(f'{path_output}/binary_recall_valid.hdf', 'w', complib='blosc', complevel=5)

        print('\nw2v_content_recall')
        print('train')
        if os.path.exists(f'{path_output}/w2v_content_recall_train.hdf'):
            w2v_content_recall_train = pd.read_hdf(f'{path_output}/w2v_content_recall_train.hdf')
        else:
            w2v_content_recall_train = w2v_concent_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, dtype='train',
                                                topn = 20, topk = 20, prefix = 'w2v')

        print('valid')
        if os.path.exists(f'{path_output}/w2v_content_recall_valid.hdf'):
            w2v_content_recall_valid = pd.read_hdf(f'{path_output}/w2v_content_recall_valid.hdf')
        else:
            w2v_content_recall_valid = w2v_concent_recall(None, inter_df, date=valid_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, dtype='train',
                                                topn=20, topk=20, prefix='w2v')
        if debug:
            w2v_content_recall_train.to_hdf(f'{path_output}/w2v_content_recall_train.hdf', 'w', complib='blosc', complevel=5)
            w2v_content_recall_valid.to_hdf(f'{path_output}/w2v_content_recall_valid.hdf', 'w', complib='blosc', complevel=5)

        # 合并召回数据
        print('\nmerge recalls')
        print('train')
        history_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        itemcf_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        binary_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        train = popular_recall_train.append(history_recall_train)
        train.drop_duplicates(subset=[uid, iid], keep='first', inplace=True)
        train = train.merge(itemcf_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(binary_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(w2v_content_recall_train, on=[uid, iid, 'label'], how='outer')

        print('valid')
        history_recall_valid.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        itemcf_recall_valid.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        binary_recall_valid.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        valid = popular_recall_valid.append(history_recall_valid)
        valid.drop_duplicates(subset=[uid, iid], keep='first', inplace=True)
        valid = valid.merge(itemcf_recall_valid, on=[uid, iid, 'label'], how='outer')
        valid = valid.merge(binary_recall_valid, on=[uid, iid, 'label'], how='outer')
        valid = valid.merge(w2v_content_recall_valid, on=[uid, iid, 'label'], how='outer')

        # 特征工程
        print('\nfeature engineer')
        print('train')
        if os.path.exists(f'{path_output}/train_fe.hdf'):
            train_fe = pd.read_hdf(f'{path_output}/train_fe.hdf')
        else:
            train_fe = feature_engineer(train, inter_df,
                                        date=train_date,
                                        user_df=user_df, item_df=item_df,
                                        uid=uid, iid=iid, time_col=time_col,
                                        last_days=7, dtype='train')
        print('valid')
        if os.path.exists(f'{path_output}/valid_fe.hdf'):
            valid_fe = pd.read_hdf(f'{path_output}/valid_fe.hdf')
        else:
            valid_fe = feature_engineer(valid, inter_df,
                                        date=valid_date,
                                        user_df=user_df, item_df=item_df,
                                        uid=uid, iid=iid, time_col=time_col,
                                        last_days=7, dtype='train')

        if debug:
            train_fe.to_hdf(f'{path_output}/train_fe.hdf', 'w', complib='blosc', complevel=5)
            valid_fe.to_hdf(f'{path_output}/valid_fe.hdf', 'w', complib='blosc', complevel=5)


        iid2idx = {}
        idx2iid = {}
        for idx, cur_iid in enumerate(train_fe[iid].unique()):
            iid2idx[cur_iid] = idx
            idx2iid[idx] = cur_iid
        self.iid2idx = iid2idx
        train_fe[iid + '_idx'] = train_fe[iid].map(iid2idx)
        valid_fe[iid + '_idx'] = valid_fe[iid].map(iid2idx)

        print(f"train_fe shape: {train_fe.shape}")
        print(f"valid_fe shape: {valid_fe.shape}")

        print('\nranker')
        # todo: 检查train_fe中是否有冗余特征, 方差为0的特征

        lgb_ranker, valid_pred = ranker(train_fe, valid_fe,
                                        uid=uid, iid=iid, time_col=time_col)

        print('\nlocal result calculation')
        # 离线结果打印
        valid_pred = valid_pred.sort_values('prob', ascending=False)
        valid_pred = valid_pred.groupby(uid).head(12).groupby(uid)[iid].agg(list).reset_index()

        begin_date = datetime.datetime.strptime(valid_date, '%Y-%m-%d %H:%M:%S') - datetime.timedelta(days=7)
        begin_date = str(begin_date)

        valid_true = inter_df.loc[inter_df[uid].isin(valid_pred[uid])]
        valid_true = valid_true[(valid_true[time_col] <= valid_date) & (valid_true[time_col] > begin_date)]

        print(valid_true[time_col].min(), valid_true[time_col].max())
        valid_true = valid_true.groupby(uid)[iid].agg(list).reset_index()

        print("mAP Score on Validation set:", mapk(valid_true[iid], valid_pred[iid]))

        self.best_iteration_ = lgb_ranker.best_iteration_


        print("#" * 30)
        print('retrain')
        # 重新训练
        train_date = valid_date
        # train_date = '2022-04-07 00:00:00'

        print('\npopular_recall')
        popular_recall_train = popular_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('\nhistory_recall')
        history_recall_train = history_recall(None, inter_df, date=train_date,
                                              uid=uid, iid=iid, time_col=time_col,
                                              last_days=7, recall_num=recall_num, dtype='train')
        print('\nitemcf_recall')
        if os.path.exists(f'{path_output}/itemcf_recall_train_all.hdf'):
            itemcf_recall_train = pd.read_hdf(f'{path_output}/itemcf_recall_train_all.hdf')
        else:
            itemcf_recall_train = itemcf_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train',
                                                topk=1000, use_iif=False, sim_last_days=14,
                                                time_decay=time_decay)

        if debug:
            itemcf_recall_train.to_hdf(f'{path_output}/itemcf_recall_train_all.hdf', 'w', complib='blosc', complevel=5)

        print('\nbinary_recall')
        if os.path.exists(f'{path_output}/binary_recall_train_all.hdf'):
            binary_recall_train = pd.read_hdf(f'{path_output}/binary_recall_train_all.hdf')
        else:
            binary_recall_train = binary_recall(None, inter_df, date=train_date,
                                                uid=uid, iid=iid, time_col=time_col,
                                                last_days=7, recall_num=recall_num, dtype='train', topk=1000)
        if debug:
            binary_recall_train.to_hdf(f'{path_output}/binary_recall_train_all.hdf', 'w', complib='blosc', complevel=5)

        print('\nw2v_content_recall')
        if os.path.exists(f'{path_output}/w2v_content_recall_train_all.hdf'):
            w2v_content_recall_train = pd.read_hdf(f'{path_output}/w2v_content_recall_train_all.hdf')
        else:
            w2v_content_recall_train = w2v_concent_recall(None, inter_df, date=train_date,
                                                          uid=uid, iid=iid, time_col=time_col,
                                                          last_days=7, dtype='train',
                                                          topn=20, topk=20, prefix='w2v')
        if debug:
            w2v_content_recall_train.to_hdf(f'{path_output}/w2v_content_recall_train_all.hdf', 'w', complib='blosc', complevel=5)


        # 合并召回数据
        print('\nmerge recalls')
        history_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        itemcf_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        binary_recall_train.drop_duplicates(subset=[uid, iid, 'label'], keep='first', inplace=True)
        train = popular_recall_train.append(history_recall_train)
        train.drop_duplicates(subset=[uid, iid], keep='first', inplace=True)
        train = train.merge(itemcf_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(binary_recall_train, on=[uid, iid, 'label'], how='outer')
        train = train.merge(w2v_content_recall_train, on=[uid, iid, 'label'], how='outer')

        # 特征工程
        print('\nfeature engineer')
        if os.path.exists(f'{path_output}/train_fe_all.hdf'):
            train_fe = pd.read_hdf(f'{path_output}/train_fe_all.hdf')
        else:
            train_fe = feature_engineer(train, inter_df,
                                        date=train_date,
                                        user_df=user_df, item_df=item_df,
                                        uid=uid, iid=iid, time_col=time_col,
                                        last_days=7, dtype='train')
        if debug:
            train_fe.to_hdf(f'{path_output}/train_fe_all.hdf', 'w', complib='blosc', complevel=5)


        train_fe[iid + '_idx'] = train_fe[iid].map(iid2idx)
        print(f"train_fe shape: {train_fe.shape}")

        print('\nranker')
        self.model, self.feats = ranker_test(train_fe, self.best_iteration_,
                                   uid=uid, iid=iid, time_col=time_col)


    def transform(self, uids):

        test_date = self.valid_date
        # test_date = '2022-04-07 00:00:00'

        print('\npopular recall, test')
        popular_recall_test = popular_recall(uids, self.inter_df, date=test_date,
                                             uid=self.uid, iid=self.iid, time_col=self.time_col,
                                             last_days=7, recall_num=self.recall_num, dtype='test')
        print('\nhistory recall, test')
        history_recall_test = history_recall(uids, self.inter_df, date=test_date,
                                             uid=self.uid, iid=self.iid, time_col=self.time_col,
                                             last_days=7, recall_num=self.recall_num, dtype='test')
        print('\nitemcf recall, test')
        if os.path.exists(f'{self.path_output}/itemcf_recall_test.hdf'):
            itemcf_recall_test = pd.read_hdf(f'{self.path_output}/itemcf_recall_test.hdf')
        else:
            itemcf_recall_test = itemcf_recall(uids, self.inter_df, date=test_date,
                                               uid=self.uid, iid=self.iid, time_col=self.time_col,
                                               last_days=7, recall_num=self.recall_num, dtype='test',
                                               topk=1000, use_iif=False, sim_last_days=14,
                                               time_decay=self.time_decay)
        if self.debug:
            itemcf_recall_test.to_hdf(f'{self.path_output}/itemcf_recall_test.hdf', 'w', complib='blosc', complevel=5)

        print('\nbinary recall, test')
        if os.path.exists(f'{self.path_output}/binary_recall_test.hdf'):
            binary_recall_test = pd.read_hdf(f'{self.path_output}/binary_recall_test.hdf')
        else:
            binary_recall_test = binary_recall(uids, self.inter_df, date=test_date,
                                               uid=self.uid, iid=self.iid, time_col=self.time_col,
                                               last_days=7, recall_num=self.recall_num, dtype='test', topk=1000)
        if self.debug:
            binary_recall_test.to_hdf(f'{self.path_output}/binary_recall_test.hdf', 'w', complib='blosc', complevel=5)

        print('\nw2v_content_recall, test')
        if os.path.exists(f'{self.path_output}/w2v_content_recall_test.hdf'):
            w2v_content_recall_test = pd.read_hdf(f'{self.path_output}/w2v_content_recall_test.hdf')
        else:
            w2v_content_recall_test = w2v_concent_recall(uids, self.inter_df, date=test_date,
                                                         uid=self.uid, iid=self.iid, time_col=self.time_col,
                                                         last_days=7, dtype='test',
                                                         topn=20, topk=20, prefix='w2v')
        if self.debug:
            w2v_content_recall_test.to_hdf(f'{self.path_output}/w2v_content_recall_test.hdf', 'w', complib='blosc', complevel=5)

        print('\nmerge recalls')
        history_recall_test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        itemcf_recall_test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        binary_recall_test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        test = popular_recall_test.append(history_recall_test)
        test.drop_duplicates(subset=[self.uid, self.iid], keep='first', inplace=True)
        test = test.merge(itemcf_recall_test, on=[self.uid, self.iid], how='outer')
        test = test.merge(binary_recall_test, on=[self.uid, self.iid], how='outer')
        test = test.merge(w2v_content_recall_test, on=[self.uid, self.iid], how='outer')

        print('\nfeature engineer')
        if os.path.exists(f'{self.path_output}/test_fe.hdf'):
            test_fe = pd.read_hdf(f'{self.path_output}/test_fe.hdf')
        else:
            test_fe = feature_engineer(test, self.inter_df,
                                       date=test_date,
                                       user_df=self.user_df, item_df=self.item_df,
                                       uid=self.uid, iid=self.iid, time_col=self.time_col,
                                       last_days=7, dtype='test')
        # if self.debug:
        #     test_fe.to_hdf(f'{self.path_output}/test_fe.hdf', 'w', complib='blosc', complevel=5)

        test_fe[self.iid + '_idx'] = test_fe[self.iid].map(self.iid2idx)
        print(f"test_fe shape: {test_fe.shape}")

        print('\ninference')
        bs = 60000
        recs = inference(self.model, self.feats, test_fe, uids,
                         uid=self.uid, iid=self.iid, time_col=self.time_col,
                         batch_size=bs)

        return recs


In [15]:
autoXRecommend = AutoXRecommend()

autoXRecommend.fit(inter_df = inter_df, user_df = None, item_df = item_df,
                  uid = uid, iid = iid, time_col = time_col,
                  recall_num = recall_num,
                  time_decay = 0.99,
                  debug = True, debug_save_path = './temp_MovieLens')


popular_recall
train
2019-11-01 00:21:32 2019-11-07 23:45:48


100%|██████████| 331/331 [00:00<00:00, 1085.05it/s]


HIT:  0.27881950171515285
valid
2019-11-08 00:01:28 2019-11-14 23:20:55


100%|██████████| 310/310 [00:00<00:00, 21320.91it/s]


HIT:  0.27728749446707573

history_recall
train
2019-11-01 00:21:32 2019-11-07 23:45:48


...

Begin training w2v model


   INFO -> estimated required memory for 11901 words and 32 dimensions: 8997156 bytes
   INFO -> resetting layer weights
   INFO -> Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2022-05-20T18:48:17.410740', 'gensim': '4.2.0', 'python': '3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]', 'platform': 'Linux-3.10.0-1127.8.2.el7.x86_64-x86_64-with-centos-7.9.2009-Core', 'event': 'build_vocab'}
   INFO -> Word2Vec lifecycle event {'msg': 'training model with 20 workers on 11901 vocabulary and 32 features, using sg=0 hs=0 sample=0.001 negative=5 window=20 shrink_windows=True', 'datetime': '2022-05-20T18:48:17.411476', 'gensim': '4.2.0', 'python': '3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]', 'platform': 'Linux-3.10.0-1127.8.2.el7.x86_64-x86_64-with-centos-7.9.2009-Core', 'event': 'train'}
   INFO -> EPOCH 0: training on 86378 raw words (81324 effective words) took 0.1s, 1144130 effective words/s
   INFO -> EPOCH 1: training on 86378 raw words (812

该循环程序运行时间： 0.75


100%|██████████| 310/310 [00:00<00:00, 930.17it/s]


w2v content recall:  (4240, 4) 0.0018867924528301887

merge recalls

feature engineer
customer feature engineer
interact feature engineer
train_fe shape: (71998, 1148)

ranker
defaultdict(<class 'dict'>, {'valid_0': {'map@12': 0.5444415510235316}})
                          feature  importance
3                      n_purchase         245
1141     latest_purchase_time_sub         216
1133  purchase_corr_item_max_time          84
4              n_purchase_nunique          73
1                    binary_score          69
0                    itemcf_score          62
1142                  movieId_idx          43
1134       purchase_corr_item_cnt          18
996                       tag_992          18
1095                     tag_1091          12
25                         tag_21          12
511                       tag_507          10
688                       tag_684          10
340                       tag_336          10
1128                     tag_1124           9
921            

In [16]:
res = autoXRecommend.transform(test_users)


popular recall, test
2019-11-08 00:01:28 2019-11-14 23:20:55


100%|██████████| 283/283 [00:00<00:00, 44956.56it/s]


history recall, test

itemcf recall, test



100%|██████████| 202/202 [01:31<00:00,  2.21it/s]



binary recall, test


...


w2v_content_recall, test
Begin training w2v model


   INFO -> deleting the raw counts dictionary of 11901 items
   INFO -> sample=0.001 downsamples 50 most-common words
   INFO -> Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 81340.72075144862 word corpus (94.2%% of prior 86378)', 'datetime': '2022-05-20T19:04:39.074374', 'gensim': '4.2.0', 'python': '3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]', 'platform': 'Linux-3.10.0-1127.8.2.el7.x86_64-x86_64-with-centos-7.9.2009-Core', 'event': 'prepare_vocab'}
   INFO -> estimated required memory for 11901 words and 32 dimensions: 8997156 bytes
   INFO -> resetting layer weights
   INFO -> Word2Vec lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2022-05-20T19:04:39.263848', 'gensim': '4.2.0', 'python': '3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]', 'platform': 'Linux-3.10.0-1127.8.2.el7.x86_64-x86_64-with-centos-7.9.2009-Core', 'event': 'build_vocab'}
   INFO -> Word2Vec lifecycle event {'msg': 'training model with 20 workers on 11901 voca

该循环程序运行时间： 0.79


100%|██████████| 202/202 [00:00<00:00, 581.23it/s]


w2v content recall:  (4020, 3)

merge recalls

feature engineer
customer feature engineer
interact feature engineer
test_fe shape: (67754, 1147)

inference
[1/1]
(67754, 1147)


## 查看结果

In [17]:
def apk(actual, predicted, k=12):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [18]:
outputs = res['prediction']
print("mAP Score on Validation set:", mapk(test_items, outputs))

mAP Score on Validation set: 0.07350586162911206


In [19]:
# mAP Score on Validation set: 0.08030425675382308
